In [2]:
import onnx
import torch
import torchvision
from pathlib import Path

In [3]:
torch.set_grad_enabled(False)

In [4]:
model = torchvision.models.__dict__['mobilenet_v2'](pretrained=False, num_classes=2)
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
model.to(device)
pytorch_total_params = sum(p.numel() for p in model.parameters())
print("pytorch_total_params:", pytorch_total_params)

pytorch_total_params: 2226434


/home/vdvu/miniconda3/envs/post-moderate/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/vdvu/miniconda3/envs/post-moderate/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


AttributeError: 'MobileNetV2' object has no attribute 'device'

In [8]:
input_sample = torch.randn(1, 3, 224, 224).to(device)
model(input_sample)

tensor([[-0.1368,  0.3174]], device='cuda:0')

In [11]:

torch.onnx.export(
    model,  # dynamic=True only compatible with cpu
    input_sample,
    'model.onnx',
    verbose=False,
    opset_version=14,
    do_constant_folding=True,  # WARNING: DNN inference with torch>=1.12 may require do_constant_folding=False
    input_names=["images"],
    output_names=["output0"],
    dynamic_axes=None,
)


In [12]:
# Checks
model_onnx = onnx.load('model.onnx')  # load onnx model
metadata = {'model': 'mobilenet_v3_small', 'athor': 'vdvu'}
# Metadata
for k, v in metadata.items():
    meta = model_onnx.metadata_props.add()
    meta.key, meta.value = k, str(v)

onnx.save(model_onnx, 'model.onnx')